In [3]:
import pprint
from typing import List
from matrix import CSRMatrix as Matrix 

# Матрица


In [4]:
a = Matrix([[1, 2], [3, 4], [5, 6]])
b = Matrix([[0, 2], [2, 0], [2, 0]])
print( b.transpose() * a , a)

c = Matrix([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]])

print(c.determinant()) # Должен быть ноль

d = Matrix([
    [2, 3, 4],
    [5, 6, 7],
    [8, 9, 10]
])
print(c.determinant())

print(d[1])


[
 [16 20]
 [ 2  4]
] [
 [1 2]
 [3 4]
 [5 6]
]
0
0
[Decimal('5'), Decimal('6'), Decimal('7')]


# Сама лаба


## Easy


### Метод гауса


In [ ]:
def gauss_solver(A: 'Matrix', b: 'Matrix') -> List['Matrix']:
    n = A.size
    augmented = A.augment(b)  # Расширенная матрица [A|b]
    
    # Прямой ход: приведение к ступенчатому виду
    rank = 0
    pivot_cols = []
    for col in range(augmented.cols - 1):  # Исключаем последний столбец (вектор b)
        # Поиск ненулевого элемента в текущем столбце
        pivot_row = -1
        for row in range(rank, n):
            if abs(augmented[row][col]) > 1e-10:
                pivot_row = row
                break
        if pivot_row == -1:
            continue  # Все элементы столбца нулевые
        
        # Перестановка строк
        augmented.swap_rows(rank, pivot_row)
        # Нормализация ведущей строки
        pivot = augmented[rank][col]
        for c in range(col, augmented.cols):
            augmented[rank][c] /= pivot
        # Исключение элементов ниже ведущего
        for row in range(rank + 1, n):
            factor = augmented[row][col]
            for c in range(col, augmented.cols):
                augmented[row][c] -= factor * augmented[rank][c]
        pivot_cols.append(col)
        rank += 1

    # Проверка на несовместность
    for row in range(rank, n):
        if abs(augmented[row][-1]) > 1e-10:
            raise ValueError("Система несовместна")

    # Определение свободных переменных
    all_vars = list(range(A.cols))
    leading_vars = pivot_cols
    free_vars = [var for var in all_vars if var not in leading_vars]
    
    # Если нет свободных переменных: единственное решение
    if not free_vars:
        x = [0] * A.cols
        for row in reversed(range(rank)):
            x[leading_vars[row]] = augmented[row][-1]
            for col in range(leading_vars[row] + 1, A.cols):
                x[leading_vars[row]] -= augmented[row][col] * x[col]
        return [Matrix([x])]

    # Построение базисных векторов
    basis = []
    for var in free_vars:
        solution = [0] * A.cols
        solution[var] = 1  # Свободная переменная = 1
        # Вычисление главных переменных
        for row in reversed(range(rank)):
            col = leading_vars[row]
            solution[col] = augmented[row][-1]
            for c in range(col + 1, A.cols):
                solution[col] -= augmented[row][c] * solution[c]
        basis.append(Matrix([solution]))
    
    return basis


In [ ]:

A = Matrix([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])
b = Matrix([[6], [15], [24]])

solutions = gauss_solver(A, b)  # Возвращает базисные векторы (если система неопределённа)

AttributeError: 'Matrix' object has no attribute 'rows'

### Центрирование данных


In [ ]:
def center_data(X: 'Matrix') -> 'Matrix':
    """
    Centers the data matrix X by subtracting the mean of each column.

    Input: data matrix X (n x m)
    Output: centered matrix X_centered (n x m)
    """
    n, m = X.size

    if n == 0 or m == 0:
        return type(X)([])

    column_means = [0.0] * m
    x_list = X.to_list()

    for row in x_list:
        for j in range(m):
            column_means[j] += float(row[j]) / n


    mean_matrix_list = [column_means[:] for _ in range(n)]
    mean_matrix = type(X)(mean_matrix_list)

    X_centered = X - mean_matrix

    return X_centered

In [ ]:
# Tests
matrix1 = Matrix([[1.0, 2.0],
         [3.0, 4.0],
         [5.0, 6.0]])

centered_matrix1 = center_data(matrix1)
print(centered_matrix1)

data2 = Matrix([[-1.0, 10.0],
         [ 0.0,  0.0],
         [ 1.0, -10.0]])

centered_matrix2 = center_data(matrix2)
print(centered_matrix2)

matrix3 = Matrix([[1, 2, 3]])
centered_matrix3 = center_data(matrix3)
print(centered_matrix3)

matrix3 = Matrix([[1, 2, 3]]).transpose()
centered_matrix3 = center_data(matrix3)
print(centered_matrix3)


[
 [-2.0 -2.0]
 [ 0.0  0.0]
 [ 2.0  2.0]
]
[
 [-1.0  10.0]
 [ 0.0   0.0]
 [ 1.0 -10.0]
]
[
 [0.0 0.0 0.0]
]
[
 [-1.0]
 [ 0.0]
 [ 1.0]
]


### Вычислить матрицу ковариаций


In [ ]:
def covariance_matrix(X_centered: 'Matrix') -> 'Matrix':
    """
    Вход: центрированная матрица X_centered (n×m)
    Выход: матрица ковариаций C (m×m)
    """
    pass

In [ ]:
# для тестов

## Normal


### Собственные значения матрицы


In [ ]:
def find_eigenvalues(C: 'Matrix', tol: float = 1e-6) -> List[float]:
    """
    Вход:
    C: матрица ковариаций (m×m)
    tol: допустимая погрешность
    Выход: список вещественных собственных значений
    """
    pass

In [ ]:
# для тестов

### Собственные векторы матрицы


In [ ]:
def find_eigenvectors(C: 'Matrix', eigenvalues: List[float]) -> List['Matrix']:
    """
    Вход:
    C: матрица ковариаций (m×m)
    eigenvalues: список собственных значений
    Выход: список собственных векторов (каждый вектор - объект Matrix)
    """
    pass

In [ ]:
# для тестов

### доля объясненной дисперсии


In [ ]:
def explained_variance_ratio(eigenvalues: List[float], k: int) -> float:
    """
    Вход:
    eigenvalues: список собственных значений
    k: число компонент
    Выход: доля объяснённой дисперсии
    """
    pass

In [ ]:
# для тестов

## Hard


### полный алгоритм PCA


In [ ]:
'''1. Центрирование данных.
2. Вычисление матрицы выборочных ковариаций.
3. Нахождение собственных значений и векторов.
4. Проекция данных на главные компоненты.'''
def pca(X: 'Matrix', k: int) -> tuple['Matrix', float]:
    """
    Вход:
    X: матрица данных (n×m)
    k: число главных компонент
    Выход:
    X_proj: проекция данных (n×k)
    : доля объяснённой дисперсии
    """
    pass

NameError: name 'Tuple' is not defined

In [ ]:
# для тестов

In [ ]:
# для тестов

### Проекция данных на две главные компоненты


In [ ]:
# Для импортов 

In [ ]:
def plot_pca_projection(X_proj: 'Matrix') -> Figure:
    """
    Вход: проекция данных X_proj (n×2)
    Выход: объект Figure из Matplotlib
    """
    pass

In [ ]:
# для тестов

### MSE


In [ ]:
def reconstruction_error(X_orig: 'Matrix', X_recon: 'Matrix') -> float:
    """
    Вход:
    X_orig: исходные данные (n×m)
    X_recon: восстановленные данные (n×m)
    Выход: среднеквадратическая ошибка MSE
    """
    pass

In [ ]:
# для тестов

## Expert


### Автоматический выбор числа главных компонент на основе порога объяснённой дисперсии


In [ ]:
def auto_select_k(eigenvalues: List[float], threshold: float = 0.95) -> int:
    """
    Вход:
    eigenvalues: список собственных значений
    threshold: порог объяснённой дисперсии
    Выход: оптимальное число главных компонент k
    """
    pass

In [ ]:
# Для тестов и результатов

### Обработать пропущенные значения в данных


In [ ]:
def handle_missing_values(X: 'Matrix') -> 'Matrix':
    """
    Вход: матрица данных X (n×m) с возможными NaN
    Выход: матрица данных X_filled (n×m) без NaN
    """
    pass

In [ ]:
# Для тестов и результатов

### Влияние шума на PCA


In [ ]:
def add_noise_and_compare(X: 'Matrix', noise_level: float = 0.1):
    """
    Вход:
    X: матрица данных (n×m)
    noise_level: уровень шума (доля от стандартного отклонения)
    Выход: результаты PCA до и после добавления шума.
    В этом задании можете проявить творческие способности, поэтому выходные данные не
    ,→ типизированы.
    """
    pass

In [ ]:
# Для тестов и результатов работы

### PCA к реальному датасету


In [ ]:
# Для импорттов

In [ ]:
# Для загрузки

In [ ]:
def apply_pca_to_dataset(dataset_name: str, k: int) -> Tuple['Matrix', float]:
    """
    Вход:
    dataset_name: название датасета
    k: число главных компонент
    Выход: кортеж (проекция данных, качество модели)
    """
    pass

In [ ]:
# Применение и результаты